In [24]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import numpy as np

hubs_dict = {
    'ARICA': 'IQUIQUE',
    'IQUIQUE': 'IQUIQUE',
    'ANTOFAGASTA': 'ANTOFAGASTA',
    'COPIAPO': 'COPIAPO',
    'COQUIMBO': 'COQUIMBO',
    'OVALLE': 'COQUIMBO',
    'ILLAPEL': 'CURAUMA',
    'LLAY LLAY': 'CURAUMA',
    'CURAUMA': 'CURAUMA',
    'SANTIAGO SUR': 'SANTIAGO SUR',
    'RANCAGUA': 'SANTIAGO SUR',
    'TALCA': 'TALCA',
    'CHILLAN': 'TALCAHUANO',
    'TALCAHUANO': 'TALCAHUANO',
    'LOS ANGELES': 'TALCAHUANO',
    'TEMUCO': 'TEMUCO',
    'VALDIVIA': 'VALDIVIA',
    'OSORNO': 'VALDIVIA',
    'PUERTO MONTT': 'PUERTO MONTT',
    'CASTRO': 'PUERTO MONTT',
    'COYHAIQUE': 'COYHAIQUE',
    'CALAMA': 'ANTOFAGASTA'
}

with open('lead_times_dict.json', 'r') as file:
    lead_times_dict = json.load(file)
    
with open('S_factor_dict.json', 'r') as file:
    S_factor_dict = json.load(file)

def create_filtered_dataframe(file, HUB, n_skus):
    datos_limpios = pd.read_csv(file)
    datos_limpios = datos_limpios.drop(columns='Unnamed: 0')
    datos_limpios_HUB = datos_limpios[datos_limpios['HUB'] == HUB]

    top_skus = datos_limpios_HUB.groupby(by=['ID_SKU_VENTA']).sum().reset_index()
    top_n_skus = top_skus.sort_values(by='Venta en pallets', ascending=False).head(n_skus)
    top_n_skus_list = list(top_n_skus['ID_SKU_VENTA'])

    datos_limpios_HUB_SKUS = datos_limpios_HUB[datos_limpios_HUB['ID_SKU_VENTA'].isin(top_n_skus_list)]
    
    return datos_limpios_HUB_SKUS


def simulation_generator(dataframe, sS_dict, sku, cd):
    
    sS_info = sS_dict[(sku, cd)]
    
    dataframe = dataframe[(dataframe['ID_SKU_VENTA'] == sku) & (dataframe['DESCR_CENDIST'] == cd)]
    
    for index, row in dataframe.iterrows():
        
        yield row['Venta en pallets'], sS_info[0], sS_info[1]
        
def stats(dataframe, confianza):

    mean_no_group = dataframe.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'Venta en pallets': 'MEDIA'})
    stdev_no_group = dataframe.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'Venta en pallets': 'STD'})
    
    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA * lead_times_dict[x['DESCR_CENDIST']], axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD * lead_times_dict[x['DESCR_CENDIST']], axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])
    
    data_completa['SS'] = data_completa.apply(lambda x: confianza * x['STD'], axis=1)

    data_completa['s'] = data_completa.apply(lambda x: x['MEDIA'] + (confianza * x['STD']), axis=1)
    
    data_completa['S'] = data_completa.apply(lambda x: x['s'] * int(S_factor_dict[str(x['ID_SKU_VENTA'])]), axis=1)
    
    return data_completa.drop(columns=['Unnamed: 0.1_x', 'Unnamed: 0.1_y'])


def create_sS_dict(dataframe):
    
    sS_dict = {}
    
    for index, row in dataframe.iterrows():
        
        sS_dict[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = (row['s'], row['S'])
        
    return sS_dict

def create_grouped_dataframe(dataframe, confianza):
    stats_no_agrupados = stats(dataframe, confianza)
    assign_hub = lambda x: hubs_dict[x['DESCR_CENDIST']]
    
    datos_para_agrupar = dataframe.copy()
    datos_para_agrupar['DESCR_CENDIST'] = datos_para_agrupar.apply(assign_hub, axis=1)
    datos_agrupados = datos_para_agrupar.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'FECHA']).sum().reset_index()
    
    stats_agrupados = stats(datos_agrupados, confianza)
    
    todos = stats_no_agrupados.merge(stats_agrupados, on=['ID_SKU_VENTA', 'DESCR_CENDIST'], how='outer').fillna(0)
    
    todos.rename(columns={'MEDIA_x': 'MEDIA_NO_AGR', 'STD_x': 'STD_NO_AGR', 'STD_y': 'STD_AGR', 'SS_x': 'SS_NO_AGR', 'SS_y': 'SS_AGR'}, inplace=True)

    todos = todos[['ID_SKU_VENTA', 'DESCR_CENDIST', 'MEDIA_NO_AGR', 'STD_NO_AGR', 'STD_AGR', 'SS_NO_AGR', 'SS_AGR']]
    
    todos.drop(columns=['STD_NO_AGR', 'STD_AGR', 'SS_NO_AGR'], inplace=True)
    
    todos['s'] = todos.apply(lambda x: x['MEDIA_NO_AGR'] + x['SS_AGR'], axis=1)
    
    todos['S'] = todos.apply(lambda x: x['s'] * int(S_factor_dict[str(x['ID_SKU_VENTA'])]), axis=1)
    
    return todos[['ID_SKU_VENTA', 'DESCR_CENDIST', 's', 'S']]
        
datos_limpios_HUB_SKUS = create_filtered_dataframe('../../data/datos_limpios_filtrados.csv', 'TALCAHUANO', 10)      

In [33]:
stats_no_agrupados = stats(datos_limpios_HUB_SKUS, 1.65)
sS_dict_no_agrupados = create_sS_dict(stats_no_agrupados)
stats_agrupados = create_grouped_dataframe(datos_limpios_HUB_SKUS, 1.65)
sS_dict_agrupados = create_sS_dict(stats_agrupados)

generador_caso_base = simulation_generator(datos_limpios_HUB_SKUS, sS_dict_no_agrupados, 450607, 'TALCAHUANO')
generador_caso_proyecto = simulation_generator(datos_limpios_HUB_SKUS, sS_dict_agrupados, 450607, 'TALCAHUANO')
    

In [34]:
print(next(generador_caso_base))
print(next(generador_caso_proyecto))

(14.95, 84.57747518771248, 338.3099007508499)
(14.95, 163.19099111236181, 652.7639644494473)


In [30]:
datos_limpios_HUB_SKUS

,Unnamed: 0.1,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en pallets,HUB
3344640,4391296,CHILLAN,622,2018-01-02,4.750,TALCAHUANO
3344641,4391297,CHILLAN,622,2018-01-03,4.000,TALCAHUANO
3344642,4391298,CHILLAN,622,2018-01-04,7.125,TALCAHUANO
3344643,4391299,CHILLAN,622,2018-01-05,11.875,TALCAHUANO
3344644,4391300,CHILLAN,622,2018-01-06,4.750,TALCAHUANO
...,...,...,...,...,...,...
16936187,18743291,TALCAHUANO,450607,2020-09-25,0.000,TALCAHUANO
16936188,18743292,TALCAHUANO,450607,2020-09-26,0.000,TALCAHUANO
16936189,18743293,TALCAHUANO,450607,2020-09-28,0.000,TALCAHUANO
16936190,18743294,TALCAHUANO,450607,2020-09-29,0.000,TALCAHUANO
